#Server

This colab notebook contains the code needed to run the models (student and general) as well as the authentication server required for the Kbot application. 

Services used are :

1. [RASA](https://rasa.com/docs/rasa)
2. [BottlePy](https://bottlepy.org/docs/dev/)
3. [Ngrok](https://ngrok.com/docs)

<br>

For ease of use with google colab the files are placed in a google drive folder.

<pre>
drive/MyDrive/.../.../ K-BOT/
└── Demo1
    ├── models
    │   ├── general.gz
    │   └── students.gz
    ├── ngrok.yml
    ├── ngrok.json
    ├── server.py 
    ├── guests.db
    └── sqlite.db  
</pre>
<center>Required Files</center>
<br>

<pre>
- Demo1      : Root folder

- models     : Contains the students.gz and general.gz files which are the trained<br>               models used by the rasa server. [R]

- sqlite.db  : sqlite database to authenticate students logging into the application 
               Contains fields (id TEXT , password TEXT) [R]

- guests.db  : sqlite database to keep track of non-student users logging in. 
               Contains fields (name TEXT,phone TEXT,logtime TIMESTAMP) [R]

- ngrok.yml  : config files for ngrok

- ngrok.json : holds ngrok generated urls for each port 

- server.py  : python server code to authenticate students

- [R]: Required files   
</pre>

<br>

The K-Bot app calls a master json file (In this case hosted on a repository in Github) which contains the sub urls (ngrok) to the auth server and the rasa server for each model. Running the cells in this notebook should update the sub-urls in the master url automatically

<pre>
{
   "url1": "https://example0.ngrok.io",
   "url2": "https://example1.ngrok.io",
   "url3": "https://example2.ngrok.io"
}
</pre>
<center>Sample structure of master json file</center>
<br><br>
To be noted that in the above shown json url1 and url2 stands for the url of the general and students model respectively whereas url3 is the link used to authenticate students. url3 is also used to log non-student users after successfull SMS/Email authentication.
<br>
<br>

Config


In [ ]:
#Path to google drive folder (mount google drive in colab first)
DRIVE_LINK = "drive/MyDrive/.../.../Demo1/*"

#Relative paths to students and general trained rasa models
STUDENT_MODEL_PATH = "models/students.gz"
GENERAL_MODEL_PATH = "models/general.gz"

#Port numbers for auth and rasa servers
AUTH_SERVER_PORT   = 5000 
STUDENT_MODEL_PORT = 5005
GENERAL_MODEL_PORT = 5006

#AuthKey for ngrok (To get one signup at https://ngrok.com)
NGROK_AUTH = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

#Personal access token for github 
GITHUBAUTH = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

#Repo containing the master url
GITHUBREPO = "<Username>/<Repo Name>"

#Relative path of master url with the repo
GITJSONLOC = "path/../../urls.json"

#Git commit message
GIT_COMMIT_MSG = "Updated json"

Installing Rasa, bottle and PyGithub

In [ ]:
!pip install -U rasa 
!pip install PyGithub
!pip install bottle

Copy files from google drive folder into the current working directory

In [ ]:
%cp -R $DRIVE_LINK .

Install ngrok

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Generate ngrok yaml config

In [ ]:
import yaml

data = dict(
    authtoken = NGROK_AUTH,
    tunnels = dict(
        general = dict(
            addr = GENERAL_MODEL_PORT,
            proto = 'http',
            bind_tls = True
        ),
        students = dict(
            addr = STUDENT_MODEL_PORT,
            proto = 'http',
            bind_tls = True
        ),
        seraph = dict(
            addr = AUTH_SERVER_PORT,
            proto = 'http',
            bind_tls = True
        )
    )
)

with open('ngrok.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

Generate server.py code

In [ ]:
%%writefile server.py
from bottle import route, run, request, response
import sqlite3
import json
import datetime

con = sqlite3.connect('sqlite.db')
cur = con.cursor()

gon = sqlite3.connect('guests.db')
gur = gon.cursor()

@route('/')
def hello():
    id = request.query.id
    password = request.query.password
    cur.execute("SELECT * FROM seraph_db WHERE id=?",[id])
    result = cur.fetchone()
    response.headers['Content-Type'] = 'application/json'
    response.set_header('Access-Control-Allow-Origin', '*')
    response.add_header('Access-Control-Allow-Methods', 'GET, POST, PUT, OPTIONS')
    if result is not None:
       if(result[1]==password):
          return json.dumps({'stat':1})
       else:
          return json.dumps({'stat':0})
    else:
       return json.dumps({'stat':0})


@route('/guest')
def guest():
  name = request.query.name
  phone = request.query.phone
  gur.execute("INSERT INTO guests VALUES(?,?,?)",[name,phone,datetime.datetime.now()])
  gon.commit()
  return str(1)

run(host='localhost', port=5000, debug=True)
con.close()
gon.close()

Writing server.py


Start server

In [ ]:
#start ngrok tunneling on configured ports
!nohup ./ngrok start -config ngrok.yml --all &

#start auth server
!nohup python server.py &

#rasa server
!nohup rasa run --model $GENERAL_MODEL_PATH --enable-api --cors "*" --port $GENERAL_MODEL_PORT &
!nohup rasa run --model $STUDENT_MODEL_PATH --enable-api --cors "*" --port $STUDENT_MODEL_PORT &

Check if process is running

In [ ]:
!ps -e -f

Keep note of the PID's of the below shown processes inorder to kill the process and restart the server incase of any errors 
<pre>
Example: 
            + ---- +                                  + ---------------------------- +
root        | 401  |     1  0 06:56 ?        00:00:18 | ./ngrok start -config ngrok. |
root        | 410  |     1  0 06:56 ?        00:00:01 | python3 server.py            |
root        | 418  |     1  0 06:56 ?        00:00:26 | /usr/bin/python3 /usr/local/ |
root        | 423  |     1  0 06:56 ?        00:00:26 | /usr/bin/python3 /usr/local/ |
            + ---- +                                  + ---------------------------- +
                ↑                                              ↑
            Example PID                                     Commands
</pre>

Having these 4 commands running helps to verify that all is fine. If any of these are defunct, check autogenerated (nohup.out) file to know more about the error produced 

To kill the process if needed

In [ ]:
# !kill -9 <PID1> <PID2> ... 
# Uncomment line 1 and replace <PID> with respective PID values to kill the process

Save ngrok urls for each port (auth server, student model, general model) into a json

In [ ]:
!curl localhost:4040/api/tunnels > ngrok.json

Visualize ngrok.json

In [ ]:
from requests import get, models
import json
from IPython.display import HTML

render_template = """
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
"""
models.Response._repr_html_ = lambda rsp: render_template % rsp.text

def render(jsondict):
  return HTML(render_template % json.dumps(jsondict))


render(json.load(open("ngrok.json")))

Initialize indexes

In [ ]:
SERAPH_INDEX  = 0
STUDENT_INDEX = 1
GENERAL_INDEX = 2

for i in range(3):
  a = json.load(open("ngrok.json"))["tunnels"][i]["name"]
  if(a == "general"):
    GENERAL_INDEX = i
  elif(a == "students"):
    STUDENT_INDEX = i
  elif(a == "seraph"):
    SERAPH_INDEX = i

print("SERAPH_INDEX   : ", SERAPH_INDEX)
print("GENERAL_INDEX  : ", STUDENT_INDEX)
print("STUDENT_INDEX  : ", GENERAL_INDEX)

SERAPH_INDEX   :  0
GENERAL_INDEX  :  2
STUDENT_INDEX  :  1


Update in Github

In [ ]:
from github import Github
import json
from datetime import datetime as dt

g = Github(GITHUBAUTH)
repo = g.get_repo(GITHUBREPO)
file = repo.get_contents(GITJSONLOC)
data = json.loads(file.decoded_content.decode("utf-8"))

data["url1"] = json.load(open("ngrok.json"))["tunnels"][GENERAL_INDEX]["public_url"]
data["url2"] = json.load(open("ngrok.json"))["tunnels"][STUDENT_INDEX]["public_url"]
data["url3"] = json.load(open("ngrok.json"))["tunnels"][SERAPH_INDEX]["public_url"]

repo.update_file(file.path, GIT_COMMIT_MSG + " at " + str(dt.now()) , json.dumps(data) , file.sha)

{'commit': Commit(sha="7b363dc4e305f0633f7877276ea69995a8d2542e"),
 'content': ContentFile(path="hosted/urls.json")}

Check if update is successful

In [ ]:
file = repo.get_contents(GITJSONLOC)
file.decoded_content

b'{"url1": "https://3517-34-105-62-241.ngrok.io", "url2": "https://ac22-34-105-62-241.ngrok.io", "url3": "https://8a55-34-105-62-241.ngrok.io"}'

Misc

In [ ]:
#Get list of guests visited

import sqlite3

guests = sqlite3.connect('guests.db')
guest_cursor = guests.cursor()

for row in guest_cursor.execute('SELECT * FROM guests'):
        print(row)

In [ ]:
#Get list of students registered

import sqlite3

students = sqlite3.connect('sqlite.db')
student_cursor = students.cursor()

for row in student_cursor.execute('SELECT * FROM seraph_db'):
        print(row)